# Juliaで方向付き丸め

## 機械区間演算

区間演算をコンピュータで実現するには$\mathbb{R}$の代わりに$\mathbb{F}$を使った区間が必要．そのような区間全体を

$$
	\mathbb{IF}:=\{\boldsymbol{x}\in\mathbb{IR}: \underline{x},~\overline{x}\in\mathbb{F}\}
$$

と定義する．IEEE754規格に準拠したシステム上では演算後の丸めの向きを制御することができる．
演算結果が浮動小数点数でない場合，丸めの向きを制御して計算する．
いま$a,b\in\mathbb{F}$に対して，$\circ\in\{+,-,\times,\div\}$として

\begin{align*}
	\mathtt{fl}_{\bigtriangledown}\!\left(a\circ b\right)&:=\max\{x\in\mathbb{F}:x\le a\circ b\}\mbox{（下向き丸め）}\\
	\mathtt{fl}_{\bigtriangleup}\!\left(a\circ b\right)&:=\min\{x\in\mathbb{F}:x\ge a\circ b\}\mbox{（上向き丸め）}
\end{align*}

とすると

$$
	\mathtt{fl}_{\bigtriangledown}\!\left(a\circ b\right)\le a\circ b\le\mathtt{fl}_{\bigtriangleup}\!\left(a\circ b\right)
$$

が成立する．

$\boldsymbol{X}=[a,b]$, $\boldsymbol{Y}=[c,d]$ ($a,b,c,d\in\mathbb{F}$)に対して，機械区間演算は次のように実現できる．

\begin{align*}
	\boldsymbol{X}+\boldsymbol{Y}&=[\mathtt{fl}_{\bigtriangledown}\!\left(a+c\right),\mathtt{fl}_{\bigtriangleup}\!\left(b+d\right)]\\
	\boldsymbol{X}-\boldsymbol{Y}&=[\mathtt{fl}_{\bigtriangledown}\!\left(a-d\right),\mathtt{fl}_{\bigtriangleup}\!\left(b-c\right)]\\
	\boldsymbol{X}\times\boldsymbol{Y}&=[\mathtt{fl}_{\bigtriangledown}\!\left(\min\{ac,ad,bc,bd\}\right),\mathtt{fl}_{\bigtriangleup}\!\left(\max\{ac,ad,bc,bd\}\right)]\\
	\boldsymbol{X}\div\boldsymbol{Y}&=[\mathtt{fl}_{\bigtriangledown}\!\left(\min\{a/c,a/d,b/c,b/d\}\right),\mathtt{fl}_{\bigtriangleup}\!\left(\max\{a/c,a/d,b/c,b/d\}\right)]
\end{align*}


|$\boldsymbol{X}\times\boldsymbol{Y}$|$c>0$|$0\in\boldsymbol{Y}$|$d<0$|
|:-------------:|:-------------:|:-------------:|:-------------:|
|$a>0$|$[\mathtt{fl}_{\bigtriangledown}\!\left(ac\right),\mathtt{fl}_{\bigtriangleup}\!\left(bd\right)]$|$[\mathtt{fl}_{\bigtriangledown}\!\left(bc\right),\mathtt{fl}_{\bigtriangleup}\!\left(bd\right)]$|$[\mathtt{fl}_{\bigtriangledown}\!\left(bc\right),\mathtt{fl}_{\bigtriangleup}\!\left(ad\right)]$|
|$0\in\boldsymbol{X}$|$[\mathtt{fl}_{\bigtriangledown}\!\left(ad\right),\mathtt{fl}_{\bigtriangleup}\!\left(bd\right)]$|$B$|$[\mathtt{fl}_{\bigtriangledown}\!\left(bc\right),\mathtt{fl}_{\bigtriangleup}\!\left(ac\right)]$|
|$b<0$|$[\mathtt{fl}_{\bigtriangledown}\!\left(ad\right),\mathtt{fl}_{\bigtriangleup}\!\left(bc\right)]$|$[\mathtt{fl}_{\bigtriangledown}\!\left(ad\right),\mathtt{fl}_{\bigtriangleup}\!\left(ad\right)]$|$[\mathtt{fl}_{\bigtriangledown}\!\left(bd\right),\mathtt{fl}_{\bigtriangleup}\!\left(ac\right)]$|

ただし $B=[\min\{\mathtt{fl}_{\bigtriangledown}\!\left(ad\right),\mathtt{fl}_{\bigtriangledown}\!\left(bc\right)\},\max\{\mathtt{fl}_{\bigtriangleup}\!\left(ad\right),\mathtt{fl}_{\bigtriangleup}\!\left(bc\right)\}].$

## ベクトル・行列の区間演算

上で述べた丸めの向きを制御することにより，ベクトル $x,y\in\mathbb{F}^n$ の内積 $x^Ty$，行列 $A, B\in\mathbb{F}^n$ の積，あるいは，ベクトル行列積 $Ax$ の結果を区間で厳密に包含することができる．

$$
	\mathtt{fl}_{\bigtriangledown}\!\left(x^Ty\right)\le x^Ty\le\mathtt{fl}_{\bigtriangleup}\!\left(x^Ty\right)
$$

$$
	\mathtt{fl}_{\bigtriangledown}\!\left(Ax\right)\le Ax\le\mathtt{fl}_{\bigtriangleup}\!\left(Ax\right)
$$

$$
	\mathtt{fl}_{\bigtriangledown}\!\left(AB\right)\le AB\le\mathtt{fl}_{\bigtriangleup}\!\left(AB\right)
$$

このようにすると丸め方向の制御で区間演算が容易にできる．しかし，行列ベクトル積，行列積を高速に実装することは職人芸のレベルの難しさである（例えば，キャシュサイズをみて最適なブロック分割などを行う）．そのため通常は数値計算ライブラリを利用するのが主流である．


Juliaの`setrounding_raw`という関数が方向付き丸めを制御できる。

```
setrounding_raw(::Type{<:Union{Float32,Float64}}, i::Integer) = ccall(:jl_set_fenv_rounding, Int32, (Int32,), i)
```

中身は単に`ccall`で`fesetround`を呼び出している。

In [1]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M2
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 2 on 4 virtual cores


In [2]:
using Pkg
Pkg.status("IntervalArithmetic")

Status `~/.julia/environments/v1.9/Project.toml`
  [d1acc4aa] IntervalArithmetic v0.20.8


## 上向き／下向き丸めをJuliaでどう制御する？

Julia v1.0.5からFloat32/Float64で[**丸めの向きを制御できない**](https://github.com/JuliaLang/julia/pull/27166)という致命的な状況になった。

現在は`setrounding(Interval, rounding_type::Symbol)`という関数で、区間演算の方向付き丸めの制御の方法を選ぶことになっている。`rounding_type::Symbol`は

- `:fast`: [FastRounding.jl](https://github.com/JeffreySarnoff/FastRounding.jl)を使った方向付き丸めの実装（無誤差変換を使うエミュレーション）、速いが極端な入力に対して**バグがある**！
- `:tight`: （デフォルト）柏木先生の「[最近点丸めのみによる方向付き丸めのエミュレート](http://verifiedby.me/kv/rounding/emu.pdf)」の実装、kvの`-DKV_NOHWROUND`オプションの実装に相当
- `:accurate`: `prevfloat`, `nextfloat` を使った丸めの実装、速いけど区間幅が大きくなる
- `:slow`: 丸めの向きを変更する実装（BigFloatで53bitに指定して、`setrounding`を使ってる？）、遅い
- `:none`: 丸めの向き変更なし（精度保証なし）、スピード比較のためのテスト用

`:fast`モードは非正規化数を扱う際に[バグ](https://github.com/JuliaIntervals/IntervalArithmetic.jl/issues/215)があるので注意が必要。

In [2]:
using IntervalArithmetic
using Pkg; Pkg.status("IntervalArithmetic")

Status `~/.julia/environments/v1.8/Project.toml`
  [d1acc4aa] IntervalArithmetic v0.20.8


In [3]:
setrounding(Interval,:fast)
tiny = interval(0, floatmin())
@show tiny * tiny
# huge = interval(floatmax(), Inf)
# @show huge * huge
# @show huge / tiny
# @show tiny / huge
setrounding(Interval,:tight)

tiny * tiny = [0, 0]


:tight

現在はデフォルトで`:tight`モードになっており、このバグは`:fast`モードで計算しない限り発生しない。

In [4]:
setrounding(Interval,:tight)
tiny = interval(0, floatmin())
tiny * tiny

[0, 4.94066e-324]

そしてJuliaの`setrounding_raw`という関数が方向付き丸めを制御できる？

```
setrounding_raw(::Type{<:Union{Float32,Float64}}, i::Integer) = ccall(:jl_set_fenv_rounding, Int32, (Int32,), i)
```

中身は単に`ccall`で`fesetround`を呼び出している。

In [5]:
xup = Base.Rounding.setrounding_raw(Float64,4194304) do # RoundUp
    parse(Float64, "0.1")# * parse(Float64, "0.2")
end
xdown = Base.Rounding.setrounding_raw(Float64,8388608) do # RoundDown
    parse(Float64, "0.1")# * parse(Float64, "0.2")
end
@show xup
@show xdown
xup > xdown

xup = 0.1
xdown = 0.09999999999999999


true

謎の整数は環境によって異なるfenvの変数で、以下のコードで分かる。

In [7]:
Base.Rounding.to_fenv(RoundUp)

4194304

In [6]:
Base.Rounding.setrounding_raw(Float64,0) # RoundNearest
x = 0.1
Base.Rounding.setrounding_raw(Float64,Base.Rounding.to_fenv(RoundUp))
@show bitstring(1.0+x)

Base.Rounding.setrounding_raw(Float64,Base.Rounding.to_fenv(RoundDown))
@show bitstring(1.0+x)

Base.Rounding.setrounding_raw(Float64,0) # RoundNearest
rounding(Float64)

bitstring(1.0 + x) = "0011111111110001100110011001100110011001100110011001100110011010"
bitstring(1.0 + x) = "0011111111110001100110011001100110011001100110011001100110011001"


RoundingMode{:Nearest}()

丸めの向きを変えて元に戻すのを忘れることがよくあるので、ある関数`f`でやりたいことを定義しておいて、
```
setrounding(f::Function, T, mode)
```
と実行できる。

In [8]:
function oneplusx()
    return bitstring(cos(x))
end

oneplusx (generic function with 1 method)

In [12]:
setrounding(oneplusx,Float64,RoundUp)

"0011111111101111110101110001001011111001101010000001011111000001"

In [9]:
setrounding(oneplusx,Float64,RoundDown)

"0011111111101111110101110001001011111001101010000001011111000000"

In [10]:
rounding(Float64)

RoundingMode{:Nearest}()

しかし、この方法は`Float64`に対しては、正式にサポートしていない。。

> Note that this is currently only supported for `T == BigFloat`.

> !!! warning
>     This function is not thread-safe. It will affect code running on all threads, but
>     its behavior is undefined if called concurrently with computations that use the
>     setting.

## 最近、Juliaで方向付き丸めが実装できる？

最近、LLVMのアップデートで`Float64`の[丸めの向きを制御できるようになった](https://github.com/JuliaLang/julia/pull/27166#issuecomment-1306908993)らしい。そこで[SetRoundingLLVM.jl](https://github.com/orkolorko/SetRoundingLLVM.jl)という怪しい（？）パッケージを使ってみる。

```julia:
(@v1.8) pkg> add https://github.com/orkolorko/SetRoundingLLVM.jl
```

と入力して、インストールする。インストールが完了したら、

```julia:
using SetRoundingLLVM
```

として呼び出す。

In [6]:
using SetRoundingLLVM
x = 0.1

llvm_setrounding(RoundUp) do
    @show bitstring(1.0+x)
end

llvm_setrounding(RoundDown) do
    @show bitstring(1.0+x)
end

bitstring(1.0 + x) = "0011111111110001100110011001100110011001100110011001100110011010"
bitstring(1.0 + x) = "0011111111110001100110011001100110011001100110011001100110011001"


"0011111111110001100110011001100110011001100110011001100110011001"

In [9]:
llvm_setrounding(oneplusx,RoundUp)

"0011111111101111110101110001001011111001101010000001011111000001"

In [10]:
llvm_setrounding(oneplusx,RoundDown)

"0011111111101111110101110001001011111001101010000001011111000000"

In [11]:
llvm_rounding()

RoundingMode{:Nearest}()

## BLASの方向付き丸め制御

ここまできたらMATLABと同様、方向付き丸めの制御をしながら、BLASを実行して高速区間演算をしたい。MATLABコードは

```MATLAB:
for n = 60:70
  disp(['n = ',num2str(n)])

  A = randn(n); B = randn(n);

  feature('setround',inf)
  AB_sup = A * B;

  feature('setround',-inf)
  AB_inf = A * B;

  feature('setround',.5)
  if ~all(AB_sup - AB_inf > 0,"all")
    break
  end
end
```

行列サイズが60〜70なのは $n>64$ でOpenBLASがマルチスレッドの計算に切り替わるサイズのようで、ここを境に[方向付き丸めが効かなくなる](https://siko1056.github.io/blog/2021/12/23/octave-matlab-directed-rounding.html)ことがよく観測されているから。

In [12]:
using LinearAlgebra
BLAS.get_config()

LinearAlgebra.BLAS.LBTConfig
Libraries: 
└ [ILP64] libopenblas64_.dylib

In [13]:
for n = 60:70
    @show n
    A, B = randn(n,n), randn(n,n)
    llvm_setrounding(RoundUp)
    Cup = A*B;
    Base.Rounding.setrounding_raw(Float64,Base.Rounding.to_fenv(RoundDown))
    Cdown = A*B;
    Base.Rounding.setrounding_raw(Float64,Base.Rounding.to_fenv(RoundNearest))
    @show all(Cup .> Cdown)
end
rounding(Float64)

n = 60
all(Cup .> Cdown) = true
n = 61
all(Cup .> Cdown) = true
n = 62
all(Cup .> Cdown) = true
n = 63
all(Cup .> Cdown) = true
n = 64
all(Cup .> Cdown) = true
n = 65
all(Cup .> Cdown) = false
n = 66
all(Cup .> Cdown) = false
n = 67
all(Cup .> Cdown) = false
n = 68
all(Cup .> Cdown) = false
n = 69
all(Cup .> Cdown) = false
n = 70
all(Cup .> Cdown) = false


RoundingMode{:Nearest}()

$n=65$で丸め向き指定する演算が失敗している。最新のLLVMの丸め向き指定をしても結果は変わらず。

In [14]:
for n = 60:70
    @show n
    A, B = randn(n,n), randn(n,n)
    Cup = llvm_setrounding(RoundUp) do
        A*B
    end
    Cdown = llvm_setrounding(RoundDown) do
        A*B
    end
    @show all(Cup .> Cdown)
end
llvm_rounding()

n = 60
all(Cup .> Cdown) = true
n = 61
all(Cup .> Cdown) = true
n = 62
all(Cup .> Cdown) = true
n = 63
all(Cup .> Cdown) = true
n = 64
all(Cup .> Cdown) = true
n = 65
all(Cup .> Cdown) = false
n = 66
all(Cup .> Cdown) = false
n = 67
all(Cup .> Cdown) = false
n = 68
all(Cup .> Cdown) = false
n = 69
all(Cup .> Cdown) = false
n = 70
all(Cup .> Cdown) = false


RoundingMode{:Nearest}()

これは予想通り。JuliaのデフォルトのOpenBLASで、方向付き丸めの制御が効かない。IntelのCPUでMKLを使っている人は方向付き丸めが変わるはず。Apple siliconの環境だと変わらない。

In [ ]:
using MKL
BLAS.get_config()

for n = 60:70
    @show n
    A, B = randn(n,n), randn(n,n)
    Cup = llvm_setrounding(RoundUp) do
        A*B
    end
    Cdown = llvm_setrounding(RoundDown) do
        A*B
    end
    @show all(Cup .> Cdown)
end
llvm_rounding()

他のBLAS?

### BLISBLAS

ごく最近、[BLIS](https://github.com/flame/blis)というBLAS-likeライブラリ（2023年のWilkinson Prize Winnerだそう）のラッパーが実装されていて、[BLISBLAS.jl](https://github.com/JuliaLinearAlgebra/BLISBLAS.jl)という名前で利用可能になっている。BLISについてはあまり良く知らないが、C99を使ったBLASの再実装のようなものだと思っている。

In [15]:
using BLISBLAS
BLAS.get_config()
# BLISBLAS.set_num_threads(8)
# BLISBLAS.get_num_threads()

LinearAlgebra.BLAS.LBTConfig
Libraries: 
├ [ILP64] libopenblas64_.dylib└ [ILP64] libblis.4.0.0.dylib

In [16]:
BLISBLAS.set_num_threads(4)

In [31]:
using AppleAccelerateLinAlgWrapper
# AppleAccelerateLinAlgWrapper.get_num_threads()

In [17]:
using AppleAccelerateLinAlgWrapper
for n = 60:70
    @show n
    A, B = randn(n,n), randn(n,n)
    Cup = llvm_setrounding(RoundUp) do
    # Cup = Base.Rounding.setrounding_raw(Float64,Base.Rounding.to_fenv(RoundUp)) do
        A*B
    end
    Cdown = llvm_setrounding(RoundDown) do
    # Cdown = Base.Rounding.setrounding_raw(Float64,Base.Rounding.to_fenv(RoundDown)) do
        A*B
    end
    @show all(Cup .> Cdown)
end
llvm_rounding()
# rounding(Float64)

n = 60
all(Cup .> Cdown) = false
n = 61
all(Cup .> Cdown) = false
n = 62
all(Cup .> Cdown) = false
n = 63
all(Cup .> Cdown) = false
n = 64
all(Cup .> Cdown) = false
n = 65
all(Cup .> Cdown) = false
n = 66
all(Cup .> Cdown) = false
n = 67
all(Cup .> Cdown) = false
n = 68
all(Cup .> Cdown) = false
n = 69
all(Cup .> Cdown) = false
n = 70
all(Cup .> Cdown) = false


RoundingMode{:Nearest}()

やっぱり変わらない。そんなに甘くはない。

### BLASの換装

最終手段、OpenBLASをmakeし直して、BLASを換装する。幸い、[libblastrampoline](https://github.com/JuliaLinearAlgebra/libblastrampoline)という謎システムのおかげでBLASの換装が簡単らしいのだが…

## 動作チェックのコード

Juliaはバージョンがコロコロ変わるので、動作をチェックするコードが必要。
以下の動作チェックのコードはいつしかの柏木先生のMATLABコード。

```
function testmm(n)
	disp('testing multiplication...')
	flag = 1;
	for k=1:n
		a=zeros(n);
		a(:,k)=ones(n,1)*1/3;
		b=zeros(n);
		b(k,:)=ones(1,n)*1/3;
		c = (rad(intval(a) * b) ~= 0) + 0.0;
		if prod(prod(c)) == 0
			disp('multiplication error!')
			flag = 0;
			break
		end
	end
	if flag == 1
		disp('multiplication OK!')
	end

	disp('testing addition...')
	flag = 1;
	for k=2:n
		a=zeros(n);
		a(:,1)=ones(n,1);
		a(:,k)=ones(n,1)*2^(-27);
		b=zeros(n);
		b(1,:)=ones(1,n);
		b(k,:)=ones(1,n)*2^(-27);
		c = (rad(intval(a) * b) ~= 0) + 0.0;
		if prod(prod(c)) == 0
			disp('addition error!')
			flag = 0;
			break
		end
	end
	if flag == 1
		disp('addition OK!')
	end
end
```


In [18]:
function testmm(n)
    println("testing multiplication...")
    flag = 1
    for k=1:n
        a=zeros(n,n)
        a[:,k] = ones(n)*1/3.
        b=zeros(n,n)
        b[k,:] = ones(n)*1/3.
        c = (radius.(map(Interval,a) * b) .!= 0) .+ 0.0
        if prod(c) == 0
            println("multiplication error!")
            flag = 0
            break
        end
    end
    if flag == 1
        println("multiplication OK!")
    end
    println("testing addition...")
    flag = 1
    for k=2:n
        a=zeros(n,n)
        a[:,1]=ones(n)
        a[:,k]=ones(n)*2^(-27)
        b=zeros(n,n)
        b[1,:]=ones(n)
        b[k,:]=ones(n)*2^(-27)
        c = (radius.(map(Interval,a) * b) .!= 0) .+ 0.0
        if prod(c) == 0
            println("addition error!")
            flag = 0
            break
        end
    end
    if flag == 1
        println("addition OK!")
    end
end

testmm (generic function with 1 method)

In [19]:
testmm(70)

testing multiplication...
multiplication OK!
testing addition...
addition OK!


In [20]:
# import Pkg; Pkg.add("IntervalMatrices")
using IntervalMatrices
# @show BLAS.vendor()
# @show BLAS.get_config()
for n = 64:65
    # n=2^i
    @show n
    A, B = IntervalMatrix(randn(n,n)), IntervalMatrix(randn(n,n))
    C = A * B
    Cup = sup(C)
    Cdown = inf(C)
    @show all(Cup .> Cdown)
end

n = 64
all(Cup .> Cdown) = false
n = 65
all(Cup .> Cdown) = false


In [21]:
function testmm_intmat(n)
    println("testing multiplication...")
    flag = 1
    for k=1:n
        a=zeros(n,n)
        a[:,k] = ones(n)*1/3.
        b=zeros(n,n)
        b[k,:] = ones(n)*1/3.
        c = (radius.(IntervalMatrix(a) * b) .!= 0) .+ 0.0
        if prod(c) == 0
            println("multiplication error!")
            flag = 0
            break
        end
    end
    if flag == 1
        println("multiplication OK!")
    end
    println("testing addition...")
    flag = 1
    for k=2:n
        a=zeros(n,n)
        a[:,1]=ones(n)
        a[:,k]=ones(n)*2^(-27)
        b=zeros(n,n)
        b[1,:]=ones(n)
        b[k,:]=ones(n)*2^(-27)
        c = (radius.(IntervalMatrix(a) * b) .!= 0) .+ 0.0
        if prod(c) == 0
            println("addition error!")
            flag = 0
            break
        end
    end
    if flag == 1
        println("addition OK!")
    end
end

testmm_intmat (generic function with 1 method)

In [22]:
testmm_intmat(70)

testing multiplication...
multiplication error!
testing addition...
addition error!


## 結論

Apple siliconのJuliaではBLASの方向付き丸めの制御はまだ早い。。でもMATLABでできているので、技術的には可能なはず。

### 謝辞

本資料も筆者が学生の頃に精度保証付き数値計算を教えて下さった[柏木雅英](http://www.kashi.info.waseda.ac.jp/~kashi/)先生の「数値解析特論」の講義資料が基になっています.
また, 以下のような文献・Web ページ等を参考にこの文章は書いています.

### 参考文献

1. 大石進一編著, 精度保証付き数値計算の基礎, コロナ社, 2018.<br>
(精度保証付き数値計算の教科書. 浮動小数点数および区間演算に詳しい. この1章が読めたら大したもの)
1. [Calculating with sets: Interval methods in Julia](https://github.com/dpsanders/IntervalsJuliaCon2020).<br>
(Juliaで区間演算をするJuliaCon2020のチュートリアル資料、[動画](https://youtu.be/LAuRCy9jUU8)も公開されている)
1. [IntervalArithmetic.jl: Basic usage](https://github.com/JuliaIntervals/IntervalArithmetic.jl/blob/master/docs/src/usage.md).<br>
(IntervalArithmetic.jlの区間演算の説明ページ)
1. matsueushi, [デフォルトの丸めモードで上付き丸め、下付き丸めをエミュレートする(Julia)](https://matsueushi.github.io/posts/rounding-emulator/).<br>
(IntervalArithmetic.jlの丸め変更はこれを使用しているようです)
1. matsueushi, [Juliaで丸めモードを指定して浮動小数点数の計算をする(したい)](https://matsueushi.github.io/posts/julia-rounding/).<br>
(丸めモードの指定ができない！？最近点丸めだけで区間演算しないといけないのか...)


<div align="right"><a href="http://www.risk.tsukuba.ac.jp/~takitoshi/">高安亮紀</a>，2020年8月6日（最終更新：2023年5月21日）</div>